In [1]:
# Note: you may need to run the following in administrator mode in Windows, esp if you had to install spacy's language models
# as admin.  See the README for details.
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
import spacy
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

import praw
import re
import random
from path import Path as path

In [25]:
# Initialize key variables here
credential_file = "credentials.key"
targetuser = "ryan_holiday" #User you are targetting, NOT necessarily your username (unless you want to do this on yourself)
test_portion = 0.1 #What fraction of the user's posts and submissions to use as the test set?
bs = 4
bptt = 70
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers
train_dir = "train/"
test_dir = "test/"

In [3]:
try:
    with open(credential_file, 'r') as f:
        creds = f.read().split('\n')
    personal = creds[0]
    secret = creds[1]
    username = creds[2]
    password = creds[3]
except OSError as e:
    print("You didn't create a credential file! Please see sample_credentials.key")
    print("Then go to http://www.storybench.org/how-to-scrape-reddit-with-python/")
    print("And register a new app named fastai_reddit in your reddit account.")
    print("And insert the values into sample_credentials.key and save it as {}.".format(credential_file))
    raise(e)

In [4]:
reddit = praw.Reddit(client_id=personal, client_secret=secret, user_agent='fastai_reddit', username=username, \
                     password=password)

In [5]:
def noquotes(text):
    return re.sub(">.+?(\n|$)","",text).replace("\\n","")

In [88]:
# Test that our Reddit connection works
subreddit = reddit.subreddit('Nootropics')
test = list(subreddit.top(limit=10))[0]
print(test.title)

We need to stop this.


In [7]:
# Test getting the latest 2 comments from the user
# See: https://www.reddit.com/r/redditdev/comments/5gt42t/praw_getting_users_last_1000_comments_causing/ 
user = reddit.redditor(targetuser)
for comment in user.comments.new(limit=2):
    print(noquotes(comment.body))

Read it however you like my friend. It's designed to be one page a day (and tied to the day's date) but it will work in just about any form. 
If you fear getting caught doing something, it's probably a sign you shouldn't be doing it. 


In [94]:
# Test getting the latest 2 submissions from the user
# See: https://www.reddit.com/r/redditdev/comments/38wzzm/praw_how_to_get_submission_text/
# TODO: write a function to remove quoted text between a ">" and a new line
for sub in user.submissions.new(limit=2):
    print(noquotes(sub.selftext))

I've just finished reading Thirteen Days as well as a number of interesting articles that attempt to "correct the record" about the Cuban Missile Crisis (including [this one from The Atlantic](https://www.theatlantic.com/magazine/archive/2013/01/the-real-cuban-missile-crisis/309190/?single_page=true)). Here is where I am confused. In Thirteen Days, RFK makes the case that Kennedy had never really wanted the Jupiter missiles in Turkey, that he had repeatedly asked for them to be removed, and possibly even thought they already had been. Therefore he was frustrated when Khrushchev demanded their removal because it was clear that the US's hardline position re: Cuba was less compelling with their existence. (Ultimately he ended up trading away their removal).

However, in a lot of contrarian histories I've read blame Kennedy for starting the crisis by placing the missiles there in 1961 (even though plans appear to predate his presidency) and more or less put the responsibility on him for pu

In [8]:
text = []
for comment in user.comments.new(limit=None):
    text.append(noquotes(comment.body))


In [96]:
print(len(text))

1000


In [9]:
subs = []
for sub in user.submissions.new(limit=None):
    newsub = noquotes(sub.selftext)
    if len(newsub) > 0:
        subs.append(newsub)

In [98]:
len(subs)

60

In [10]:
alltext = text + subs
random.shuffle(alltext)
split = round(test_portion*len(alltext))
test = alltext[0:split]
train = alltext[split:]
train_p = path(train_dir)
test_p = path(test_dir)
train_p.mkdir_p()
test_p.mkdir_p()
for f in train_p.files() + test_p.files():
    f.remove()

In [16]:
for i,t in enumerate(train):
    with open(train_dir + str(i) + ".txt", "w") as f:
        f.write(str(str(t).encode('ascii', 'ignore')))

for i,t in enumerate(test):
    with open(test_dir + str(i) + ".txt", "w") as f:
        f.write(str(str(t).encode('ascii', 'ignore')))

In [17]:
len(train), len(test)

(954, 106)

In [18]:
TEXTS = data.Field(lower=True, tokenize="spacy")
FILES = dict(train=train_dir, validation=test_dir, test=test_dir)

In [26]:
md = LanguageModelData.from_text_files("./", TEXTS, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [22]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [27]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [28]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                                                                                         
    0      4.647748   4.522188  
    1      4.626875   5.155661                                                                                         
    2      4.626266   4.556951                                                                                         
    3      4.640644   4.533238                                                                                         
    4      4.626163   4.516031                                                                                         
    5      4.603915   4.516233                                                                                         
    6      4.612507   4.519444                                                                                         
    7      4.634662   4.525259                                                                                         
    8  

[array([4.51606])]